## Land Surface Temperature

- **Install packages and Authenticate to Earth Engine**

In [ ]:
!pip install earthengine-api #Earth-engine Python API
!pip install geopandas  #needed for shapefile import

In [ ]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=SEzf3Gv0aAg5YEfTBbBHaNBGz6zPesLMremPmAvn1ls&tc=LxVA_N0QwNcOlNHrvJE31ymLvFeORAWhcr-sKrICwHQ&cc=NOBF-Pk5Qpu5e_0L-28toi31LfOps2IRI8ks254eLBo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BUw2G_2AVQvzovOeJxpO7NfrV3jf4wmHzTNgK77Tgb6KQLnuv3T9js

Successfully saved authorization token.




```
# This is formatted as code
```

**Authenticate to Google Drive. We need this for exporting rasters**


In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Mounted at /content/drive


In [ ]:
# import packages
import os
import geopandas as gpd
import time
from random import randint
import ee

#Initialize EE
ee.Initialize()

**Set paths**


In [ ]:
main_dir=r"/content/drive/My Drive/az/GEE_main_folder"
# cities = r"{}/Inputs/NodalPourashavasInPolygonShariatpurFixed.shp".format(main_dir) # path to the cities shapfile that has been uploaded to Gdrive
cities = r"{}/Inputs/compiled_clusters_plus_cities.shp".format(main_dir) # path to the cities shapfile that has been uploaded to Gdrive
cities_reprojected = gpd.read_file(cities).to_crs({'init': 'epsg:4326'})
cities_reprojected["city_name"]=cities_reprojected["Name"].str.replace("'",'')
cities_reprojected["city_name"]=cities_reprojected["city_name"].str.replace(" ","_")
cities_reprojected["city_name"]=cities_reprojected["city_name"].str.lower()
cities = cities_reprojected["city_name"].unique()
cities=cities.tolist()

cities_reprojected

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,OBJECTID_1,OBJECTID,Name,CentX,CentY,Name_1,Name_12,Thana,District,Division,...,Geocode_T,Area_skm,Id,ORIG_FID,Shape_Leng,Shape_Le_1,Shape_Area,city_name,Cluster,geometry
0,1.0,33.0,Benapole,88.901558,23.046581,Benapole,Benapole,Sharsha,Jessore,Khulna,...,40419008,9.064438,0,0.0,0.000000,0.189363,0.000799,benapole,NaN,"POLYGON ((88.90239 23.06093, 88.90259 23.06086..."
1,2.0,73.0,Cox's Bazar,91.991353,21.437164,Coxbazar,Coxbazar,Coxbazar Sadar,Cox's Bazar,Chittagong,...,20222466,20.914755,0,0.0,0.000000,0.330361,0.001812,coxs_bazar,NaN,"POLYGON ((91.97055 21.46849, 91.97058 21.46849..."
2,3.0,137.0,Jessore,89.202501,23.160782,Jessore,Jessore,Kotwali,Jessore,Khulna,...,40414716,24.961994,0,0.0,0.000000,0.354204,0.002200,jessore,NaN,"POLYGON ((89.17218 23.19600, 89.17234 23.19594..."
3,4.0,178.0,Kushtia,89.130603,23.899518,Kushtia,Kushtia,Kushtia Sadar,Kushtia,Khulna,...,40507960,11.960377,0,0.0,0.000000,0.164902,0.001060,kushtia,NaN,"POLYGON ((89.12187 23.91669, 89.12196 23.91663..."
4,5.0,188.0,Madhabdi,90.674309,23.851744,Madhabdi,Madhabdi,Narsingdi Sadar,Narsingdi,Dhaka,...,30686025,4.620399,0,0.0,0.000000,0.117562,0.000408,madhabdi,NaN,"POLYGON ((90.66131 23.85987, 90.66349 23.86083..."
5,6.0,246.0,Panchagarh,88.563864,26.327681,Panchagarh,Panchagarh,Panchagarh Sadar,Panchagarh,Rangpur,...,55777350,10.308067,0,0.0,0.000000,0.207683,0.000932,panchagarh,NaN,"POLYGON ((88.58409 26.34643, 88.58418 26.34639..."
6,7.0,274.0,Saidpur,88.886533,25.785489,Saidpur,Saidpur,Saidpur,Nilphamari,Rangpur,...,55738575,22.931918,0,0.0,0.000000,0.500144,0.002064,saidpur,NaN,"POLYGON ((88.87179 25.81681, 88.87191 25.81665..."
7,8.0,48.0,Mirsharai,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0,0.0,32430.732237,0.304486,0.001448,mirsharai,NaN,"POLYGON ((91.57307 22.78600, 91.57331 22.78589..."
8,9.0,69.0,Feni,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0,0.0,22384.343508,0.208677,0.001931,feni,NaN,"POLYGON ((91.40700 23.04207, 91.40775 23.04172..."
9,10.0,84.0,Madaripur,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0,0.0,24473.298340,0.232160,0.001256,madaripur,NaN,"POLYGON ((90.16742 23.18511, 90.16746 23.18475..."


**Define function to export LST rasters**


In [ ]:
def export_landsurface_temperature( country , cities_reprojected, max_value , years, month0 ,  month1):

    """
    export_landsurface_temperature: Exports Land Surface temperature rasters for a given country in the cities shapefile

    :param country: Name of the country
    :param cities_reprojected: geopandas read cities shapefile
    :param max_value: cap the number of cities
    :param years: years
    :param month0 : first hottest month
    :param month1 : last hottest month

    :return: confirms the export of rasters

    """

    # Create a jsondictionary from the geometry of the  shapefile
    geom = cities_reprojected['geometry']
    jsonDict = eval(geom.to_json())

    for index, row in cities_reprojected.iterrows():
        try:
          city= row['city_name'].lower() #instruct the column for city name in the city shapefile
          city= city.replace("'","")
          if index  <= max_value:
              city_number= index
              for x in jsonDict['features'][city_number]['geometry']['coordinates']:
                  AOI = ee.Geometry.Polygon(x) #cast polygon as ee geometry
                  landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

                  # Filter for hottest months in the past X years
                  def filter_hot_month(i):
                      return ee.Filter.date(years[i] + '-' + month0 + '-01', years[i] + '-' + month1 + '-01')

                  range_list= list(map(filter_hot_month, list(range(0, 10)))) #combination of months and years
                  rangefilter = ee.Filter.Or(range_list)

                  # Define a function to scale the data and mask unwanted pixels
                  def maskL457sr(image):
                      # Bit 0 - Fill
                      # Bit 1 - Dilated Cloud
                      # Bit 2 - Cirrus (high confidence)
                      # Bit 3 - Cloud
                      # Bit 4 - Cloud Shadow
                      qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
                      saturationMask = image.select('QA_RADSAT').eq(0)
                      # Apply the scaling factors to the appropriate bands.
                      thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)
                      # Replace the original bands with the scaled ones and apply the masks.
                      return image.addBands(thermalBand, None, True).updateMask(qaMask).updateMask(saturationMask)

                  # Apply filter and mask
                  collectionSummer = landsat.filter(rangefilter).filterBounds(AOI).map(maskL457sr).select('ST_B10').mean().add(-273.15).clip(AOI)
                  task = ee.batch.Export.image.toDrive(**{
                      'image': collectionSummer,
                      'description': city + "_Summer",
                      'folder': country,
                      'region': AOI,
                      'scale': 30,
                      'maxPixels': 1e9
                  })
                  task.start()

        except Exception as e:
            print(f"Error : {e} ")
    return  print(f"Successfully exported the rasters for: {country}")

**Set Args and call the function**


In [ ]:
#Args
max_value=len(cities_reprojected.Name)
# max_value=5
years =list(str(i) for i in range(2013, 2024)) #years from 2013-2023
month0 = '04'  # first hottest month  # update for each country. For bangladesh its April
month1 = '07'  # end of hottest month (note that this is exclusive)  # update for each country:For bangladesh its June but insert 7 because of the range function
country= 'BGD' # replace with the relevent country name
start_exporting= export_landsurface_temperature(country, cities_reprojected, max_value, years, month0, month1)

Successfully exported the rasters for: BGD
